# 교차검증 & 튜닝

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [2]:
cancer_data = load_breast_cancer()

In [ ]:
cancer_data.data

In [ ]:
cancer_data.target

In [5]:
# 폴드 세트 설정
k = 5

In [7]:
X = cancer_data.data
y = cancer_data.target

In [6]:
cancer_data.data.shape, cancer_data.target.shape

((569, 30), (569,))

In [10]:
# 총 데이터 갯수를 k로 나눔 (한 블록이 몇개를 가질 것인지) -> 올림으로 데이터 전체 다 쓸 수 있게
import math
block_size = math.ceil(X.shape[0]/k)
block_size

114

In [17]:
# K-fold 로직
model = RandomForestClassifier()
score_list = []

for i in range(k):
    # testset에 대한 index는 block_size*idx:block_size*(idx+1) 여기에 포함이 되어있음
    # 데이터 개수 vs block_size로 만든 맨 마지막 index값 중 작은 것을 선택 (안그러면 index out of range 에러 발생)
    max_length = min(block_size*(i+1), X.shape[0])
    test_idxs = range(block_size*i, max_length)
    # print(test_idxs)

    # train index는 어떻게 만들어야 할까?
    # 전체 데이터 개수로 이루어진 index 모음에서 test index에 포함되지 않은 것만 가져오기
    # train_idxs = [idx for idx in range(X.shape[0]) if idx not in test_idxs]

    train_idxs = []
    # 데이터 개수만큼 idx를 반복 0 ~ 데이터 개수
    for idx in range(X.shape[0]):
        # idx가 test_idxs에 포함되어있지 않으면,
        if idx not in test_idxs:
            # train idxs에 넣기
            train_idxs.append(idx)

    # print(train_idxs)

    train_X, train_y = X[train_idxs], y[train_idxs]
    # print(train_X.shape) # print(train_y.shape)

    test_X, test_y = X[test_idxs], y[test_idxs]
    # print(test_X.shape) # print(test_y.shape)

    # 모델 학습
    model.fit(train_X, train_y)

    # 모델 평가
    score = model.score(test_X, test_y)
    score_list.append(score)

avg_score = sum(score_list) / len(score_list) # = k
print(avg_score)

0.9578481602235678


In [26]:
def kfold_train(estimator, X, y, k=5):
    score_list = []

    for i in range(k):
        max_length = min(block_size*(i+1), X.shape[0])
        test_idxs = range(block_size*i, max_length)

        train_idxs = [idx for idx in range(X.shape[0]) if idx not in test_idxs]

        train_X, train_y = X[train_idxs], y[train_idxs]

        test_X, test_y = X[test_idxs], y[test_idxs]

        model.fit(train_X, train_y)
        score = model.score(test_X, test_y)
        score_list.append(score)
        
    avg_score = sum(score_list) / k # = k
    return avg_score

In [27]:
model = RandomForestClassifier()
kfold_train(model, cancer_data.data, cancer_data.target, k=5)

0.9590643274853802

In [29]:
# GridSearchCV
grid_parameters = {'max_depth': [1,2,3], 'min_samples_split': [2, 3]}
cv = 3

In [31]:
# 가능한 모든 조합을 찾아준다.
from itertools import product
list(product([1,2,3], [4,5]))

[(1, 4), (1, 5), (2, 4), (2, 5), (3, 4), (3, 5)]

In [34]:
# 각각의 값들을 key들과 합쳐준다.
param = dict(zip(grid_parameters, (1, 4)))
param

{'max_depth': 1, 'min_samples_split': 4}

In [35]:
model = RandomForestClassifier(**param)

In [36]:
model

RandomForestClassifier(max_depth=1, min_samples_split=4)

In [38]:
score = kfold_train(model, cancer_data.data, cancer_data.target, k=cv)

In [39]:
print('현재 파라미터', str(param))
print('현재 정확도', score)

현재 파라미터 {'max_depth': 1, 'min_samples_split': 4}
현재 정확도 0.8888888888888888


In [43]:
list(product(grid_parameters.values()))

[([1, 2, 3],), ([2, 3],)]

In [45]:
for value in list(product(*grid_parameters.values())):
    param = dict(zip(grid_parameters, value))
    param

    model = RandomForestClassifier(**param)
    score = kfold_train(model, cancer_data.data, cancer_data.target, k=cv)

    print('현재 파라미터', str(param))
    print('현재 정확도', score)

현재 파라미터 {'max_depth': 1, 'min_samples_split': 2}
현재 정확도 0.8918128654970761
현재 파라미터 {'max_depth': 1, 'min_samples_split': 3}
현재 정확도 0.8859649122807017
현재 파라미터 {'max_depth': 2, 'min_samples_split': 2}
현재 정확도 0.9298245614035089
현재 파라미터 {'max_depth': 2, 'min_samples_split': 3}
현재 정확도 0.9298245614035089
현재 파라미터 {'max_depth': 3, 'min_samples_split': 2}
현재 정확도 0.9385964912280702
현재 파라미터 {'max_depth': 3, 'min_samples_split': 3}
현재 정확도 0.9356725146198831


In [46]:
def gridsearch_cv(model_class, grid_parameters, data, target, cv=3):
    grid_list = []
    for value in list(product(*grid_parameters.values())):
        param = dict(zip(grid_parameters, value))
        param

        model = model_class(**param)
        score = kfold_train(model, data, target, k=cv)
        
        grid_list.append([param, score])
    return grid_list

In [49]:
result = gridsearch_cv(RandomForestClassifier, grid_parameters, cancer_data.data, cancer_data.target, cv=5)

In [50]:
result

[[{'max_depth': 1, 'min_samples_split': 2}, 0.9455674584691819],
 [{'max_depth': 1, 'min_samples_split': 3}, 0.9473218444340941],
 [{'max_depth': 2, 'min_samples_split': 2}, 0.9508306163639186],
 [{'max_depth': 2, 'min_samples_split': 3}, 0.9438130725042695],
 [{'max_depth': 3, 'min_samples_split': 2}, 0.9526005278683435],
 [{'max_depth': 3, 'min_samples_split': 3}, 0.9490917559385188]]